In [4]:
import pandas as pd
import yfinance as yf
import pyodbc
from datetime import datetime

# Define tickers in a list
tickers = ["AMD", "NVDA", "AMZN", "VOO", "PLTR", "TSLA", "AAPL"]

# Define interval for fetching data
interval = "1d"

data = yf.download(tickers, period="5y", interval=interval)
df = data.stack(level=1, future_stack=True).reset_index()

df.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
df = df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

[*********************100%***********************]  7 of 7 completed


"\ndf.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']\ndf = df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]\nprint(df)\n"

In [2]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Kiyoworld;"
    "DATABASE=USStocks;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()
cursor.fast_executemany = True

try :
    insert_query = "INSERT INTO raw_us_stocks (Ticker, Date, [Open], high, low, [Close], volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    
    print("Loading Raw Data into SQL Server...")
    
    rows = list(df.itertuples(index=False, name=None))
    cursor.executemany(insert_query, rows)
    conn.commit()
    print(f"Successfully inserted {len(df)} records.") 

except Exception as e :
    print(f"Error : {e}")

Loading Raw Data into SQL Server...
Successfully inserted 8785 records.


In [3]:
try :
    cursor.execute('EXEC sp_Load_Staging_Layer')
    conn.commit()
    print('Success')
except Exception as e :
    print(f'Error : {e}')
finally:
    cursor.close()
    conn.close()

Success
